In [1]:
import json

In [47]:
with open('questions.json', 'r') as file:
    data = json.load(file)

    if isinstance(data, dict):
        print("Keys:", list(data.keys()))

    data = data['data']

Keys: ['version', 'data']


In [ ]:
data[400]['paragraphs'][1]['qas'][1]['answers'][0].get('text')

In [83]:
data[400]['paragraphs'][1]['qas'][1].get('answers')

[{'text': 'the Church holds that Mary was also sinless personally, "free from all sin, original or personal',
  'answer_start': 160}]

In [113]:
gooz = []

In [114]:
import pandas as pd

for i in range(len(data)):
    paragraphs = data[i]['paragraphs']
    for j in range(len(paragraphs)):
        qas = paragraphs[j]['qas']
        for k in range(len(qas)):
            question = qas[k]['question']
            answer = qas[k].get('answers')
            if len(answer) > 0:
                # text = answer
                completion = answer[0].get('text')
            if question != '':
                prompt = question
            gooz.append({"prompt": question, "completion": completion})
            # text = "Q: " + prompt + " A: " + completion
            # gooz.append(text)



In [ ]:
gooz[::100]

In [126]:
import pandas as pd

df = pd.DataFrame(gooz)

In [132]:
df['completion'].str.len().max()

239

In [136]:
file_path = "qa_dataset.json1"

try:
    with open(file_path, 'w') as json_file:
        json.dump(gooz, json_file, indent=4)
    print(f"Dictionary successfully saved to {file_path}")
except IOError as e:
    print(f"Error saving file: {e}")

Dictionary successfully saved to qa_dataset.json1


In [207]:
import json

with open("qa_dataset.json", "r") as f_in, open("qa_dataset.jsonl", "w") as f_out:
    data = json.load(f_in)  # Loads the list of objects
    for item in data:
        json.dump(item, f_out)
        f_out.write('\n')  # Write each object on its own line

In [24]:
import json

# n = 0
with open("datasets/alpaca_data.json", "r") as f_in, open("datasets/alpaca_data.jsonl", "w") as f_out:
    data = json.load(f_in)
    for item in data:
        temp = {}
        if item['input']:
            temp['prompt'] = f"{item['instruction']}\n\n{item['input']}"
        else:
            temp['prompt'] = item['instruction']
        temp['completion'] = item['output']
        json.dump(temp, f_out)
        f_out.write('\n')
        # n += 1
        # if n > 10:
        #     break;

In [142]:
import tiktoken

encoder = tiktoken.get_encoding('gpt2')
encoder.encode("<BOS>")[0]

27

In [149]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# tokenizer.get_vocab()
tokenizer.encode("Hello my friend")

[15496, 616, 1545]

In [150]:
special_tokens_dict = {
    "bos_token": "<BOS>",
    "eos_token": "<EOS>",
    "sep_token": "<SEP>",
    "pad_token": "<PAD>"
}
tokenizer.add_special_tokens(special_tokens_dict)

4

In [502]:
# len(tokenizer.get_vocab())
# tokenizer.get_vocab()["<PAD>"]
# tokenizer.encode("<PAD>")
tokenizer.decode([27, 2215, 750, 37361, 344, 923, 5033, 2968, 30, 27, 259, 262, 2739, 6303, 82, 27])

'<When did Beyonce start becoming popular?<in the late 1990s<'

In [25]:
import json
import random
from transformers import GPT2Tokenizer

class QADataLoader:
    def __init__(self, filepath, max_length=512, shuffle=True):
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.max_length = max_length
        self.shuffle = shuffle

        self.tokenizer.add_special_tokens({
            "bos_token": "<BOS>",
            "eos_token": "<EOS>",
            "sep_token": "<SEP>",
            "pad_token": "<PAD>"
        })

        self.special_tokens = {
            "<BOS>": self.tokenizer.encode("<BOS>")[0],
            "<SEP>": self.tokenizer.encode("<SEP>")[0],
            "<EOS>": self.tokenizer.encode("<EOS>")[0],
            "<PAD>": self.tokenizer.encode("<PAD>")[0]
        }

        self.samples = []
        with open(filepath, 'r') as f:
            for line in f:
                item = json.loads(line.strip())
                q, a = item["prompt"], item["completion"]
                tokens = self.encode_sample(q, a)
                if len(tokens["input_ids"]) <= self.max_length:
                    self.samples.append(tokens)

    def encode_sample(self, question, answer):
        q_tokens = self.tokenizer.encode(question)
        a_tokens = self.tokenizer.encode(answer)

        input_ids = (
            [self.special_tokens["<BOS>"]] + 
            q_tokens + 
            [self.special_tokens["<SEP>"]] +
            a_tokens +
            [self.special_tokens["<EOS>"]]
        )

        label_ids = (
            [-100] * (1 + len(q_tokens) + 1) + 
            a_tokens +
            [self.special_tokens["<EOS>"]]
         )

        return {"input_ids": input_ids, "label_ids": label_ids}     


    def __len__(self):
        return len(self.samples)

    def get_batch(self, batch_size):
        if self.shuffle:
            batch = random.sample(self.samples, batch_size)
        else:
            batch = self.samples[:batch_size]
        
        max_len = max(len(sample["input_ids"]) for sample in batch)
        input_ids_batch = []
        label_ids_batch = []
        attention_mask_batch = []

        for sample in batch:
            pad_len = max_len - len(sample["input_ids"])
            input_ids = sample["input_ids"] + [self.special_tokens["<PAD>"]] * pad_len
            label_ids = sample["label_ids"] + [-100] * pad_len
            attention_mask = [1] * len(sample["input_ids"]) + [0] * pad_len

            input_ids_batch.append(input_ids)
            label_ids_batch.append(label_ids)
            attention_mask_batch.append(attention_mask)
            
        return {
            "input_ids": input_ids_batch, 
            "label_ids": label_ids_batch,
            "attention_mask": attention_mask_batch
        }     

/Users/reza/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [732]:
dataloader = QADataLoader("qa_small_dataset.jsonl", max_length=512)

In [736]:
len(dataloader.samples) // 2

25

In [699]:
batches = dataloader.get_batch(2)
# print(batches['input_ids'][0])
# print(batches['label_ids'][0])
# tokenizer.decode(batches['inputs_ids'][0])

# batches['input_ids'][0]
tokenizer.decode(batches['input_ids'][0])

batches['attention_mask']
# batches
# print(tokenizer.decode(batches["input_ids"][3]))

# tokenizer.decode(batches['label_ids'][0])
# batches['label_ids'][0]
# tokenizer.decode(-100)


# for batch in batches:
#     print(len(batch['input_ids']))
#     print(len(batch['label_ids']))

attention_mask = batches['attention_mask']
attention_mask = torch.tensor(attention_mask)
attention_mask[:, None, None, :].shape

torch.Size([2, 1, 1, 21])

In [700]:
attention_mask.shape

torch.Size([2, 21])

In [189]:
import numpy as np
gooz_ids = np.random.randint(0, 10, size=20).tolist()
gooz_labels = gooz_ids[:10]
gooz_labels = gooz_labels + [-100] * 10
gooz_ids, gooz_labels


([9, 6, 7, 7, 7, 9, 6, 0, 1, 0, 6, 8, 6, 7, 6, 2, 0, 6, 3, 9],
 [9,
  6,
  7,
  7,
  7,
  9,
  6,
  0,
  1,
  0,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100])

In [714]:
import torch
import torch.nn as nn

B, T, n_embd = 2, 4, 8
n_head = 2

bias = torch.tril(torch.ones(T, T)).view(1, 1, T, T)

x = torch.randn(B, T, n_embd)

c_attn = nn.Linear(n_embd, 3*n_embd)
qkv = c_attn(x)
q, k, v = qkv.split(n_embd, dim=2)

k = k.view(B, T, n_head, n_embd // n_head).transpose(1, 2) # (B, nh, T, hs)
q = q.view(B, T, n_head, n_embd // n_head).transpose(1, 2) # (B, nh, T, hs)
v = v.view(B, T, n_head, n_embd // n_head).transpose(1, 2) # (B, nh, T, hs)

q.shape

att = q @ k.transpose(-2, -1)
att.shape, bias.shape
att = att.masked_fill(bias[:, :, :T, :T] == 0, float('inf'))
bias[:, :, :T, :T]
att

tensor([[[[-6.2602e-01,         inf,         inf,         inf],
          [ 1.2883e-01, -5.3285e-02,         inf,         inf],
          [-4.2524e-01,  1.2064e-01, -5.7070e-01,         inf],
          [ 1.7759e+00,  9.9432e-01,  4.1436e-04,  7.3606e-01]],

         [[ 4.9656e-01,         inf,         inf,         inf],
          [ 9.4131e-03, -4.7672e-02,         inf,         inf],
          [ 2.0352e-01,  3.7636e-01,  1.4296e-01,         inf],
          [-5.0593e-01, -5.2762e-01, -8.0086e-01, -1.4362e-02]]],


        [[[-5.1723e-02,         inf,         inf,         inf],
          [ 3.5109e-01, -2.1513e-01,         inf,         inf],
          [ 1.0144e+00, -2.6050e-01,  4.5848e-01,         inf],
          [ 2.3837e-01, -3.6955e-01, -3.2761e-01, -5.8813e-01]],

         [[ 2.8595e-01,         inf,         inf,         inf],
          [ 2.5033e-01, -1.2565e-01,         inf,         inf],
          [-6.0656e-01,  1.5357e-01,  1.4696e-01,         inf],
          [-8.0639e-01,  2.7095e

In [702]:
attention_mask = attention_mask[:, None, None, :]
att = att.masked_fill(attention_mask == 0, float('inf'))
att

RuntimeError: The size of tensor a (21) must match the size of tensor b (16) at non-singleton dimension 3

In [731]:
x.shape
x1 = x[-1,-1, :]
x1[:1]

tensor([0.8019])